##  Installation and Setup

In [21]:
# Install required packages
!pip install -q openai transformers torch sqlite3 requests python-dotenv huggingface_hub


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


## Import Libraries and Configuration

In [2]:
import sqlite3
import json
import re
import datetime
import os
from typing import Dict, List, Any, Optional
import requests
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# Try to import optional packages
try:
    import openai
    HAS_OPENAI = True
except ImportError:
    HAS_OPENAI = False
    print("⚠️ OpenAI not available")

try:
    from transformers import pipeline
    HAS_TRANSFORMERS = True
except ImportError:
    HAS_TRANSFORMERS = False
    print("⚠️ Transformers not available")

# Configuration
@dataclass
class Config:
    """Configuration class for the AI agent"""
    database_path: str = "ai_agent.db"
    openai_api_key: str = " "  # Set your OpenAI API key here if you have one
    use_local_llm: bool = True  # Use local model fallback

config = Config()
print("✅ Configuration loaded successfully!")

✅ Configuration loaded successfully!


## Database Manager Class

In [3]:
class DatabaseManager:
    """Handles all database operations for the AI agent"""

    def __init__(self, db_path: str):
        self.db_path = db_path
        self.init_database()
        self.populate_sample_data()

    def init_database(self):
        """Initialize the database with required tables"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Create chat_memory table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS chat_memory (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_input TEXT NOT NULL,
                agent_response TEXT NOT NULL,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')

        # Create products table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS products (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                category TEXT NOT NULL,
                price DECIMAL(10,2) NOT NULL
            )
        ''')

        # Create sales table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS sales (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                product_id INTEGER NOT NULL,
                quantity INTEGER NOT NULL,
                sale_date DATETIME DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (product_id) REFERENCES products (id)
            )
        ''')

        conn.commit()
        conn.close()
        print("✅ Database initialized successfully")

    def populate_sample_data(self):
        """Populate database with sample data"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Check if data already exists
        cursor.execute("SELECT COUNT(*) FROM products")
        if cursor.fetchone()[0] > 0:
            conn.close()
            return

        # Sample products
        products = [
            ('Laptop A', 'Electronics', 1000.00),
            ('Laptop B', 'Electronics', 1200.00),
            ('Mouse Pro', 'Electronics', 50.00),
            ('Keyboard X', 'Electronics', 80.00),
            ('Monitor 4K', 'Electronics', 300.00),
            ('Tablet Y', 'Electronics', 400.00),
            ('Smartphone Z', 'Electronics', 800.00),
            ('Headphones', 'Electronics', 150.00)
        ]

        cursor.executemany(
            "INSERT INTO products (name, category, price) VALUES (?, ?, ?)",
            products
        )

        # Sample sales
        sales = [
            (1, 5, '2024-01-15 10:00:00'),  # Laptop A
            (1, 3, '2024-01-20 14:30:00'),  # Laptop A
            (2, 2, '2024-01-25 09:15:00'),  # Laptop B
            (3, 10, '2024-02-01 11:45:00'), # Mouse Pro
            (4, 8, '2024-02-05 16:20:00'),  # Keyboard X
            (5, 4, '2024-02-10 13:10:00'),  # Monitor 4K
            (1, 2, '2024-02-15 12:00:00'),  # Laptop A
            (6, 6, '2024-02-20 15:30:00'),  # Tablet Y
            (7, 3, '2024-02-25 10:45:00'),  # Smartphone Z
            (8, 12, '2024-03-01 14:15:00')  # Headphones
        ]

        cursor.executemany(
            "INSERT INTO sales (product_id, quantity, sale_date) VALUES (?, ?, ?)",
            sales
        )

        conn.commit()
        conn.close()
        print("✅ Sample data populated successfully")

    def store_conversation(self, user_input: str, agent_response: str):
        """Store conversation in chat memory"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute(
            "INSERT INTO chat_memory (user_input, agent_response) VALUES (?, ?)",
            (user_input, agent_response)
        )

        conn.commit()
        conn.close()

    def get_recent_conversations(self, limit: int = 5) -> List[Dict]:
        """Retrieve recent conversations for context"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute(
            "SELECT user_input, agent_response, timestamp FROM chat_memory ORDER BY timestamp DESC LIMIT ?",
            (limit,)
        )

        conversations = []
        for row in cursor.fetchall():
            conversations.append({
                'user_input': row[0],
                'agent_response': row[1],
                'timestamp': row[2]
            })

        conn.close()
        return conversations

    def execute_sql_query(self, query: str) -> List[Dict]:
        """Execute SQL query and return results"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        try:
            cursor.execute(query)

            # Get column names
            columns = [description[0] for description in cursor.description]

            # Fetch results
            results = cursor.fetchall()

            # Convert to list of dictionaries
            result_dicts = []
            for row in results:
                result_dicts.append(dict(zip(columns, row)))

            conn.close()
            return result_dicts

        except Exception as e:
            conn.close()
            raise Exception(f"SQL Error: {str(e)}")

# Test the DatabaseManager
db_manager = DatabaseManager(config.database_path)
print("✅ DatabaseManager created and tested!")

✅ Database initialized successfully
✅ Sample data populated successfully
✅ DatabaseManager created and tested!


## Calculator Tool

In [4]:
class CalculatorTool:
    """Calculator tool for mathematical operations"""

    @staticmethod
    def calculate(expression: str) -> str:
        """Safely evaluate mathematical expressions"""
        try:
            # Handle percentage calculations
            if '%' in expression:
                return CalculatorTool._handle_percentage(expression)

            # Clean the expression - only allow safe characters
            expression = re.sub(r'[^0-9+\-*/().\s]', '', expression)

            # Evaluate safely
            result = eval(expression)
            return f"Result: {result}"

        except Exception as e:
            return f"Error calculating: {str(e)}"

    @staticmethod
    def _handle_percentage(expression: str) -> str:
        """Handle percentage calculations like '25% of 1600'"""
        # Pattern: "X% of Y"
        pattern = r'(\d+(?:\.\d+)?)%\s*of\s*(\d+(?:\.\d+)?)'
        match = re.search(pattern, expression.lower())

        if match:
            percentage = float(match.group(1))
            value = float(match.group(2))
            result = (percentage / 100) * value
            return f"{percentage}% of {value} = {result}"

        return "Invalid percentage format. Use 'X% of Y'"

# Test the Calculator
calc = CalculatorTool()
print("Testing Calculator:")
print(calc.calculate("25% of 1600"))
print(calc.calculate("15 + 30 * 2"))
print("✅ Calculator tool ready!")

Testing Calculator:
25.0% of 1600.0 = 400.0
Result: 75
✅ Calculator tool ready!


## Web Search Tool

In [5]:
class WebSearchTool:
    """Web search tool (simulated for Colab environment)"""

    @staticmethod
    def search(query: str) -> str:
        """Simulate web search results"""
        # Predefined responses for common queries
        search_responses = {
            "latest ai frameworks": """Latest AI Frameworks (2025):
1. TensorFlow 3.0 - Enhanced performance and usability
2. PyTorch 2.0 - Improved compilation and speed
3. LangChain - For LLM application development
4. LlamaIndex - Data framework for LLM applications
5. Hugging Face Transformers - Pre-trained models
6. JAX - High-performance ML research
7. FastAPI - For AI API development
8. Streamlit - AI app deployment
9. AutoGPT - Autonomous AI agents
10. CrewAI - Multi-agent AI systems""",

            "ai tools": """Popular AI Tools (2025):
- OpenAI GPT-4 - Advanced language model
- Claude 3.5 - Anthropic's AI assistant
- Midjourney - AI image generation
- Stable Diffusion - Open-source image AI
- GitHub Copilot - AI code assistant
- Cursor - AI-powered code editor
- Perplexity - AI search engine
- Notion AI - AI-powered productivity
- Jasper - AI writing assistant
- Runway - AI video generation""",

            "machine learning": """Machine Learning Trends (2025):
- Foundation Models and Large Language Models
- Multimodal AI (text, image, audio)
- Edge AI deployment
- AI Agents and Autonomous Systems
- Retrieval-Augmented Generation (RAG)
- Fine-tuning and LoRA techniques
- AI Safety and Alignment
- Federated Learning
- Neural Architecture Search
- Quantum Machine Learning"""
        }

        # Find best match
        query_lower = query.lower()
        for key, response in search_responses.items():
            if key in query_lower:
                return f"🔍 Search Results for '{query}':\n\n{response}"

        # Default response
        return f"🔍 Search Results for '{query}':\n\nSimulated search results would appear here. In a real implementation, this would connect to a search API like Google Custom Search or Bing Search API."

# Test the Web Search
search_tool = WebSearchTool()
print("Testing Web Search:")
print(search_tool.search("latest ai frameworks"))
print("✅ Web search tool ready!")

Testing Web Search:
🔍 Search Results for 'latest ai frameworks':

Latest AI Frameworks (2025):
1. TensorFlow 3.0 - Enhanced performance and usability
2. PyTorch 2.0 - Improved compilation and speed
3. LangChain - For LLM application development
4. LlamaIndex - Data framework for LLM applications
5. Hugging Face Transformers - Pre-trained models
6. JAX - High-performance ML research
7. FastAPI - For AI API development
8. Streamlit - AI app deployment
9. AutoGPT - Autonomous AI agents
10. CrewAI - Multi-agent AI systems
✅ Web search tool ready!


## SQL Tool

In [6]:
class SQLTool:
    """Tool for generating and executing SQL queries"""

    def __init__(self, db_manager: DatabaseManager):
        self.db_manager = db_manager

    def generate_sql_query(self, natural_language_query: str) -> str:
        """Generate SQL query from natural language"""
        query_lower = natural_language_query.lower()

        # Sales-related queries
        if "total sales" in query_lower:
            if "laptop a" in query_lower:
                return """
                SELECT
                    p.name,
                    SUM(s.quantity * p.price) as total_sales,
                    SUM(s.quantity) as total_quantity
                FROM sales s
                JOIN products p ON s.product_id = p.id
                WHERE LOWER(p.name) = 'laptop a'
                GROUP BY p.name
                """
            elif "all products" in query_lower or "total sales" in query_lower:
                return """
                SELECT
                    p.name,
                    SUM(s.quantity * p.price) as total_sales,
                    SUM(s.quantity) as total_quantity
                FROM sales s
                JOIN products p ON s.product_id = p.id
                GROUP BY p.name
                ORDER BY total_sales DESC
                """

        # Product listing queries
        if "products" in query_lower and ("list" in query_lower or "show" in query_lower):
            return "SELECT * FROM products ORDER BY category, name"

        # Price queries
        if "price" in query_lower:
            return "SELECT name, category, price FROM products ORDER BY price DESC"

        # Category queries
        if "category" in query_lower or "electronics" in query_lower:
            return "SELECT name, price FROM products WHERE LOWER(category) = 'electronics' ORDER BY price DESC"

        # Recent sales
        if "recent sales" in query_lower:
            return """
            SELECT
                p.name,
                s.quantity,
                s.sale_date,
                (s.quantity * p.price) as sale_value
            FROM sales s
            JOIN products p ON s.product_id = p.id
            ORDER BY s.sale_date DESC
            LIMIT 10
            """

        return None

    def execute_query(self, natural_language_query: str) -> str:
        """Execute SQL query based on natural language"""
        try:
            sql_query = self.generate_sql_query(natural_language_query)

            if not sql_query:
                return "I couldn't generate a SQL query for that request. Please try rephrasing your question."

            results = self.db_manager.execute_sql_query(sql_query.strip())

            if not results:
                return "No results found for your query."

            # Format results
            return self._format_results(results, natural_language_query)

        except Exception as e:
            return f"Error executing query: {str(e)}"

    def _format_results(self, results: List[Dict], original_query: str) -> str:
        """Format query results for display"""
        if not results:
            return "No results found."

        # Format based on query type
        if "total sales" in original_query.lower():
            formatted = "📊 Sales Summary:\n\n"
            for row in results:
                formatted += f"• {row.get('name', 'Product')}: ${row.get('total_sales', 0):,.2f} "
                formatted += f"({row.get('total_quantity', 0)} units)\n"
            return formatted

        elif "recent sales" in original_query.lower():
            formatted = "📈 Recent Sales:\n\n"
            for row in results:
                formatted += f"• {row.get('name', 'Product')}: {row.get('quantity', 0)} units "
                formatted += f"on {row.get('sale_date', 'N/A')} (${row.get('sale_value', 0):,.2f})\n"
            return formatted

        else:
            # Generic formatting
            formatted = "📋 Query Results:\n\n"
            for i, row in enumerate(results, 1):
                formatted += f"{i}. "
                for key, value in row.items():
                    formatted += f"{key}: {value}, "
                formatted = formatted.rstrip(", ") + "\n"
            return formatted

# Test the SQL Tool
sql_tool = SQLTool(db_manager)
print("Testing SQL Tool:")
print(sql_tool.execute_query("What are total sales for Laptop A?"))
print("✅ SQL tool ready!")

Testing SQL Tool:
📊 Sales Summary:

• Laptop A: $10,000.00 (10 units)

✅ SQL tool ready!


## LLM Manager

In [14]:
class LLMManager:
    """Manages LLM interactions for the AI agent"""

    def __init__(self, config: Config):
        self.config = config
        self.llm = None
        self.setup_llm()

    def setup_llm(self):
        """Setup LLM based on configuration"""
        if self.config.use_local_llm and HAS_TRANSFORMERS:
            try:
                # Use a simple text generation pipeline
                self.llm = pipeline("text-generation",
                                  model="gpt2",
                                  max_length=100,
                                  device=-1)  # Use CPU
                print("✅ Local LLM (GPT-2) initialized successfully")
            except Exception as e:
                print(f"❌ Failed to initialize local LLM: {e}")
                self.llm = None
        else:
            if self.config.openai_api_key and HAS_OPENAI:
                openai.api_key = self.config.openai_api_key
                print("✅ OpenAI API initialized")
            else:
                print("⚠️ No LLM available - using fallback responses")

    def classify_intent(self, user_input: str) -> str:
        """Classify user intent based on input"""
        user_input_lower = user_input.lower()

        # Mathematical operations
        if any(op in user_input_lower for op in ['+', '-', '*', '/', '%', 'calculate', 'what is']):
            if '%' in user_input_lower and 'of' in user_input_lower:
                return "calculator"
            elif any(char.isdigit() for char in user_input):
                return "calculator"

        # Search queries
        if any(keyword in user_input_lower for keyword in ['search', 'find', 'look up', 'latest']):
            return "search"

        # SQL/Database queries
        if any(keyword in user_input_lower for keyword in ['sales', 'product', 'total', 'price', 'category']):
            return "sql"

        # Greetings
        if any(greeting in user_input_lower for greeting in ['hi', 'hello', 'hey', 'good morning', 'good afternoon']):
            return "greeting"

        # Follow-up questions
        if any(phrase in user_input_lower for phrase in ['that total', 'what about', 'and that', 'the previous']):
            return "followup"

        # Default to general chat
        return "chat"

    def generate_response(self, user_input: str, context: str = "") -> str:
        """Generate response using LLM"""
        if self.llm:
            try:
                prompt = f"User: {user_input}\nAssistant:"
                if context:
                    prompt = f"Context: {context}\n{prompt}"

                response = self.llm(prompt, max_length=len(prompt) + 50,
                                  do_sample=True, temperature=0.7)

                # Extract generated text
                generated_text = response[0]['generated_text']
                assistant_response = generated_text.split("Assistant:")[-1].strip()

                return assistant_response if assistant_response else "I'm here to help!"

            except Exception as e:
                return f"I'm here to help! (LLM error: {str(e)})"

        # Fallback responses
        fallback_responses = {
            "greeting": "Hello! I'm your AI assistant. I can help you with calculations, searches, and database queries. What would you like to know?",
            "thanks": "You're welcome! Is there anything else I can help you with?",
            "default": "I'm here to help! You can ask me to calculate something, search for information, or query our sales database."
        }

        user_lower = user_input.lower()
        if any(greeting in user_lower for greeting in ['hi', 'hello', 'hey']):
            return fallback_responses["greeting"]
        elif any(thanks in user_lower for thanks in ['thank', 'thanks']):
            return fallback_responses["thanks"]
        else:
            return fallback_responses["default"]

# Test the LLM Manager
llm_manager = LLMManager(config)
print("Testing LLM Manager:")
print("Intent:", llm_manager.classify_intent("What's 25% of 1600?"))
print("Response:", llm_manager.generate_response("Hi there!"))
print("✅ LLM Manager ready!")

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


✅ Local LLM (GPT-2) initialized successfully
Testing LLM Manager:
Intent: calculator
Response: No. I was in
✅ LLM Manager ready!


## Main AI Agent Class

In [15]:
class AIAgent:
    """Main AI Agent class that orchestrates all components"""

    def __init__(self, config: Config):
        self.config = config
        self.db_manager = DatabaseManager(config.database_path)
        self.llm_manager = LLMManager(config)
        self.calculator = CalculatorTool()
        self.web_search = WebSearchTool()
        self.sql_tool = SQLTool(self.db_manager)
        print("🤖 AI Agent initialized successfully!")

    def process_query(self, user_input: str) -> str:
        """Process user query and generate response"""
        # Classify intent
        intent = self.llm_manager.classify_intent(user_input)

        # Generate response based on intent
        response = self._handle_intent(user_input, intent)

        # Store conversation in database
        self.db_manager.store_conversation(user_input, response)

        return response

    def _handle_intent(self, user_input: str, intent: str) -> str:
        """Handle different types of user intents"""

        if intent == "calculator":
            return self.calculator.calculate(user_input)

        elif intent == "search":
            # Extract search query
            search_query = self._extract_search_query(user_input)
            return self.web_search.search(search_query)

        elif intent == "sql":
            return self.sql_tool.execute_query(user_input)

        elif intent == "followup":
            return self._handle_followup(user_input)

        elif intent == "greeting":
            return "Hello! I'm your AI assistant. I can help you with:\n" + \
                   "• 🧮 Calculations (e.g., 'What's 25% of 1600?')\n" + \
                   "• 🔍 Search queries (e.g., 'Search latest AI frameworks')\n" + \
                   "• 📊 Sales data queries (e.g., 'What are total sales for Laptop A?')\n" + \
                   "• 💬 General conversation\n\nWhat would you like to know?"

        else:
            # Use LLM for general chat
            return self.llm_manager.generate_response(user_input)

    def _extract_search_query(self, user_input: str) -> str:
        """Extract search query from user input"""
        # Remove common search prefixes
        prefixes = ['search for', 'search', 'find', 'look up', 'what is']
        query = user_input.lower()

        for prefix in prefixes:
            if query.startswith(prefix):
                query = query[len(prefix):].strip()
                break

        return query or user_input

    def _handle_followup(self, user_input: str) -> str:
        """Handle follow-up questions using conversation history"""
        recent_conversations = self.db_manager.get_recent_conversations(limit=3)

        if not recent_conversations:
            return "I don't have any previous conversation to refer to."

        # Look for numerical values in recent responses
        for conv in recent_conversations:
            response = conv['agent_response']
            if any(keyword in response.lower() for keyword in ['total', 'sales', '$', 'result']):
                return f"📋 Previous Result:\n{response}"

        return "I'm not sure what you're referring to. Could you be more specific?"

    def show_help(self):
        """Show help information"""
        help_text = """
📚 AVAILABLE COMMANDS:

🧮 CALCULATOR:
- "What's 25% of 1600?"
- "Calculate 15 + 30 * 2"
- "35% of 1400"

🔍 SEARCH:
- "Search latest AI frameworks"
- "Find AI tools"
- "Look up machine learning trends"

📊 DATABASE QUERIES:
- "What are total sales for Laptop A?"
- "Show all products"
- "List recent sales"
- "What's the price of products?"

💬 GENERAL:
- "Hi" or "Hello"
- "What about that total again?" (follow-up)
        """
        print(help_text)

# Initialize the AI Agent
agent = AIAgent(config)

✅ Database initialized successfully


Device set to use cpu


✅ Local LLM (GPT-2) initialized successfully
🤖 AI Agent initialized successfully!


## Test the Agent

In [16]:
# Test queries
test_queries = [
    "Hi",
    "What's 25% of 1600?",
    "What are total sales for Laptop A?",
    "Search latest AI frameworks",
    "Show all products",
    "What about that total again?"
]

print("🧪 TESTING AI AGENT")
print("=" * 50)

for query in test_queries:
    print(f"\n👤 Test Query: {query}")
    response = agent.process_query(query)
    print(f"🤖 Agent Response: {response}")
    print("-" * 50)

print("\n✅ All tests completed!")

🧪 TESTING AI AGENT

👤 Test Query: Hi
🤖 Agent Response: Hello! I'm your AI assistant. I can help you with:
• 🧮 Calculations (e.g., 'What's 25% of 1600?')
• 🔍 Search queries (e.g., 'Search latest AI frameworks')
• 📊 Sales data queries (e.g., 'What are total sales for Laptop A?')
• 💬 General conversation

What would you like to know?
--------------------------------------------------

👤 Test Query: What's 25% of 1600?
🤖 Agent Response: 25.0% of 1600.0 = 400.0
--------------------------------------------------

👤 Test Query: What are total sales for Laptop A?
🤖 Agent Response: 📊 Sales Summary:

• Laptop A: $10,000.00 (10 units)

--------------------------------------------------

👤 Test Query: Search latest AI frameworks
🤖 Agent Response: 🔍 Search Results for 'latest ai frameworks':

Latest AI Frameworks (2025):
1. TensorFlow 3.0 - Enhanced performance and usability
2. PyTorch 2.0 - Improved compilation and speed
3. LangChain - For LLM application development
4. LlamaIndex - Data framework

##  Interactive Chat Function

In [17]:
def start_interactive_chat():
    """Start interactive chat session"""
    print("\n" + "="*50)
    print("🤖 AI AGENT CHAT SESSION")
    print("="*50)
    print("Type 'quit' or 'exit' to end the session")
    print("Type 'help' for available commands")
    print("-"*50)

    while True:
        try:
            user_input = input("\n👤 You: ").strip()

            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("👋 Goodbye! Thank you for using the AI Agent.")
                break

            if user_input.lower() == 'help':
                agent.show_help()
                continue

            if not user_input:
                continue

            # Process query and get response
            response = agent.process_query(user_input)
            print(f"\n🤖 Agent: {response}")

        except KeyboardInterrupt:
            print("\n\n👋 Session ended by user. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {str(e)}")

# Uncomment the line below to start interactive chat
# start_interactive_chat()

print("🎉 AI Agent is ready!")
print("Run 'start_interactive_chat()' to begin chatting!")

🎉 AI Agent is ready!
Run 'start_interactive_chat()' to begin chatting!


## Quick Test Functions

In [18]:
# Quick test functions you can run
def quick_test():
    """Run a quick test of all features"""
    print("🔧 Quick Test of All Features:")
    print("-" * 40)

    # Test calculator
    print("1. Calculator Test:")
    print(agent.process_query("What's 30% of 500?"))

    print("\n2. Search Test:")
    print(agent.process_query("Search AI tools"))

    print("\n3. Database Test:")
    print(agent.process_query("What are total sales for Laptop A?"))

    print("\n4. Follow-up Test:")
    print(agent.process_query("What about that total again?"))

def show_database_contents():
    """Show what's in the database"""
    print("📊 Database Contents:")
    print("-" * 30)

    # Show products
    results = agent.db_manager.execute_sql_query("SELECT * FROM products")
    print("Products:")
    for product in results:
        print(f"  {product['name']} - ${product['price']}")

    # Show recent conversations
    conversations = agent.db_manager.get_recent_conversations(3)
    print(f"\nRecent Conversations ({len(conversations)}):")
    for conv in conversations:
        print(f"  User: {conv['user_input'][:50]}...")

# Run quick test
quick_test()

# Show database contents
show_database_contents()

🔧 Quick Test of All Features:
----------------------------------------
1. Calculator Test:
30.0% of 500.0 = 150.0

2. Search Test:
🔍 Search Results for 'ai tools':

Popular AI Tools (2025):
- OpenAI GPT-4 - Advanced language model
- Claude 3.5 - Anthropic's AI assistant
- Midjourney - AI image generation
- Stable Diffusion - Open-source image AI
- GitHub Copilot - AI code assistant
- Cursor - AI-powered code editor
- Perplexity - AI search engine
- Notion AI - AI-powered productivity
- Jasper - AI writing assistant
- Runway - AI video generation

3. Database Test:
📊 Sales Summary:

• Laptop A: $10,000.00 (10 units)


4. Follow-up Test:
I couldn't generate a SQL query for that request. Please try rephrasing your question.
📊 Database Contents:
------------------------------
Products:
  Laptop A - $1000
  Laptop B - $1200
  Mouse Pro - $50
  Keyboard X - $80
  Monitor 4K - $300
  Tablet Y - $400
  Smartphone Z - $800
  Headphones - $150

Recent Conversations (3):
  User: What's 25% of 160

In [19]:
# Quick test of all features
quick_test()

# Start interactive chat
start_interactive_chat()



🔧 Quick Test of All Features:
----------------------------------------
1. Calculator Test:
30.0% of 500.0 = 150.0

2. Search Test:
🔍 Search Results for 'ai tools':

Popular AI Tools (2025):
- OpenAI GPT-4 - Advanced language model
- Claude 3.5 - Anthropic's AI assistant
- Midjourney - AI image generation
- Stable Diffusion - Open-source image AI
- GitHub Copilot - AI code assistant
- Cursor - AI-powered code editor
- Perplexity - AI search engine
- Notion AI - AI-powered productivity
- Jasper - AI writing assistant
- Runway - AI video generation

3. Database Test:
📊 Sales Summary:

• Laptop A: $10,000.00 (10 units)


4. Follow-up Test:
I couldn't generate a SQL query for that request. Please try rephrasing your question.

🤖 AI AGENT CHAT SESSION
Type 'quit' or 'exit' to end the session
Type 'help' for available commands
--------------------------------------------------

👤 You: Search latest AI frameworks

🤖 Agent: 🔍 Search Results for 'latest ai frameworks':

Latest AI Frameworks (202

In [22]:
def show_architecture():
    """Display the system architecture"""
    architecture = """
🏗️ AI AGENT ARCHITECTURE

📦 COMPONENTS:
├── DatabaseManager - SQLite database operations
├── LLMManager - Language model integration
├── CalculatorTool - Mathematical operations
├── WebSearchTool - Search functionality (simulated)
├── SQLTool - Natural language to SQL conversion
└── AIAgent - Main orchestrator

🔄 WORKFLOW:
User Input → Intent Classification → Tool Selection → Processing → Response

📊 DATABASE SCHEMA:
├── chat_memory (conversations)
├── products (product catalog)
└── sales (transaction data)

🎯 CAPABILITIES:
✅ Natural language understanding
✅ Mathematical calculations (including percentages)
✅ Simulated web search
✅ SQL query generation and execution
✅ Conversation memory
✅ Context-aware follow-ups
    """
    print(architecture)

show_architecture()

print("\n🎉 AI Agent Setup Complete!")
print("=" * 50)
print("✅ Database initialized with sample data")
print("✅ All tools configured and tested")
print("✅ Conversation memory enabled")
print("\n🚀 Ready to use! Try running:")
print("• quick_test() - Test all features")
print("• start_interactive_chat() - Start chatting")
print("• agent.process_query('your question') - Single query")


🏗️ AI AGENT ARCHITECTURE

📦 COMPONENTS:
├── DatabaseManager - SQLite database operations
├── LLMManager - Language model integration  
├── CalculatorTool - Mathematical operations
├── WebSearchTool - Search functionality (simulated)
├── SQLTool - Natural language to SQL conversion
└── AIAgent - Main orchestrator

🔄 WORKFLOW:
User Input → Intent Classification → Tool Selection → Processing → Response

📊 DATABASE SCHEMA:
├── chat_memory (conversations)
├── products (product catalog)  
└── sales (transaction data)

🎯 CAPABILITIES:
✅ Natural language understanding
✅ Mathematical calculations (including percentages)
✅ Simulated web search
✅ SQL query generation and execution
✅ Conversation memory
✅ Context-aware follow-ups
    

🎉 AI Agent Setup Complete!
✅ Database initialized with sample data
✅ All tools configured and tested
✅ Conversation memory enabled

🚀 Ready to use! Try running:
• quick_test() - Test all features
• start_interactive_chat() - Start chatting
• agent.process_query('yo